In [ ]:
!pip install langchain
!pip install openai
!pip install pinecone-client
!pip install tiktoken
!pip install asyncio

In [2]:
import os
os.environ['OPENAI_API_KEY'] = "abc"
os.environ['PINECONE_API_KEY'] = "123"

## Prepare FAISS index using Pinecone

In [3]:
import os
import asyncio
import tiktoken
import pinecone
from IPython.display import Markdown, display
from langchain.embeddings import OpenAIEmbeddings, HypotheticalDocumentEmbedder
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.vectorstores import Pinecone
pinecone.init(api_key=os.environ['PINECONE_API_KEY'], environment="us-west1-gcp")
embeddings = OpenAIEmbeddings()

In [ ]:
# Create Pinecone FAISS Index
pinecone.create_index("health", dimension=1536)

In [ ]:
# Split text file into chunks
with open('webmd.txt') as f:
    harrisons = f.read()
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_text(harrisons)

In [ ]:
# Upload chunks to Pinecone
async def upsertTexts(arr):
    return Pinecone(pinecone.Index("health"), embeddings.embed_query, "text").add_texts(arr)

In [ ]:
batch_size = 100
for i in range(0, len(texts), batch_size):
    await upsertTexts(texts[i:i+batch_size])

## Query index as a chatbot with memory

In [49]:
template = """You are a doctor having a conversation with a patient.

Given the following extracted parts of a long document and a question, create a final factual answer.

{context}

{chat_history}
Patient: {human_input}
Doctor:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input", ai_prefix="Doctor", human_prefix="Patient")
docsearch = Pinecone(pinecone.Index("health"), embeddings.embed_query, "text")
stop = ["Doctor:", "Patient:", "The following", "Other things"]
llm = OpenAI(model_name="gpt-4", temperature=0, stop=stop)
chain = load_qa_chain(llm, chain_type="stuff", memory=memory, prompt=prompt)

In [50]:
def askDoctor(query):
    docs = docsearch.similarity_search(query)[0:2]
    return chain({'input_documents': docs, 'human_input': query})['output_text'].strip()

In [51]:
askDoctor("I feel light headed all of a sudden and I'm unable to walk for a short time. What is the cause?")

'Fainting can be caused by a variety of things. The most common cause is a drop in blood pressure. This can happen for a variety of reasons, including anxiety, fear, pain, intense emotional stress, hunger, or use of alcohol or drugs. Most people who suffer from simple fainting have no underlying heart or neurological (nerve or brain) problem.\n\n \n\nSome people have a problem with the way their body regulates their blood pressure, particularly when they move too quickly from a lying or sitting position to a standing position. This condition is called postural hypotension and may be severe enough to cause fainting. This type of fainting is more common in the elderly, people who recently had a lengthy illness that kept them in bed and people who have poor muscle tone.'

In [52]:
askDoctor("I'm also unable to close my hand into a fist. What could it be?")

"The inability to close your hand into a fist could be the result of a variety of conditions, including:\n\n \n\n- Arthritis\n- Tendonitis\n- Broken bones\n- Sprains\n- Strains\n- Nerve damage\n- Parkinson's disease\n- Stroke\n- Diabetes\n- Thyroid disease\n- Parkinson's disease\n- Stroke\n- Diabetes\n- Thyroid disease"

In [53]:
askDoctor("What can I do to treat it in the short term?")

'For immediate relief, apply a cold pack or ice wrapped in a cloth to the painful area.'

In [44]:
memory

ConversationBufferMemory(human_prefix='Patient', ai_prefix='Doctor', buffer="\nPatient: I feel light headed all of a sudden and I'm unable to walk for a short time. What is the cause?\nDoctor:  \n\nFainting can be caused by a variety of things. The most common cause is a drop in blood pressure. This can happen for a variety of reasons, including anxiety, fear, pain, intense emotional stress, hunger, or use of alcohol or drugs. Most people who suffer from simple fainting have no underlying heart or neurological (nerve or brain) problem.\n\n \n\nSome people have a problem with the way their body regulates their blood pressure, particularly when they move too quickly from a lying or sitting position to a standing position. This condition is called postural hypotension and may be severe enough to cause fainting. This type of fainting is more common in the elderly, people who recently had a lengthy illness that kept them in bed and people who have poor muscle tone.\n\n \n\n\nPatient: I'm al